In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, f1_score
from sklearn.linear_model import LinearRegression
import math
from itertools import combinations
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


# Load your dataset

## datatset 

 https://www.kaggle.com/datasets/kartik2112/fraud-detection



In [ ]:
# importing the train and test data sets
data1 = pd.read_csv('/kaggle/input/fraud-detection/fraudTest.csv')
data2 = pd.read_csv('/kaggle/input/fraud-detection/fraudTrain.csv')

In [ ]:
# concatanating the two data sets together
data = pd.concat([data1, data2])

In [ ]:
data = data.sample(frac=1, random_state=1).reset_index()
data = data.head(n=100000)
data.is_fraud.value_counts()

In [ ]:
# doing some prelimenary adjustments to data set
data = data.drop('Unnamed: 0', axis=1)
data = data.drop('trans_num', axis=1)
data = data.drop('first', axis=1)
data = data.drop('last', axis=1)
data = data.drop('street', axis=1)
data = data.drop('city', axis=1)


In [ ]:
# calculating distance between credit card holder location and location of merchant
data['distance'] = np.sqrt((data['lat'] - data['merch_lat'])**2 + (data['long'] - data['merch_long'])**2)

# converting to date time
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])

In [ ]:
# pulling the hour for a variable
def pull_hour(ts):
    return ts.hour
data['hour'] = data['trans_date_trans_time'].apply(pull_hour)

In [ ]:
# using unix time we are going to calculate the sum of transaction amounts in past 30 days
# we will create two variables from this, first sum of transactions in 30 days
# next will be a interaction based variable between current purchase amount / 30 day total
# this will help measure if this transaction is out of the ordinary



# function to calculate last 30 day spending
def sum_30_day(unixtime, cc_num):
    unixstamp = unixtime
    minus30 = unixstamp - 2629743
    ccnum = cc_num
    sumtable = data.loc[(data["cc_num"] == ccnum) & (data['unix_time'] < unixstamp) & (data['unix_time'] > minus30)]
    history30 = sumtable['amt'].sum()
    return history30

    
# running function and creating a new variable for it
data['history_30'] = data.apply(lambda x: sum_30_day(x.unix_time, x.cc_num), axis=1)

# measuring interaction effect with amt in new variable
data['interaction_30'] = data['history_30'] / data['amt']

In [ ]:
# dropping non categorical variables in preperation for regression modeling

data = data.drop('trans_date_trans_time', axis=1)
data = data.drop('state', axis=1)
data = data.drop('merchant', axis=1)
data = data.drop('job', axis=1)
data = data.drop('dob', axis=1)
data = data.drop('category', axis=1)
data = data.drop('gender', axis=1)
data = data.drop('index', axis=1)


In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(data.corr(),annot=True).set_title('Correlation')

In [ ]:
# there is multicollinearity issues with our non generated predictors such as lat longs, we will drop all of these

data = data.drop('cc_num', axis=1)
data = data.drop('zip', axis=1)
data = data.drop('lat', axis=1)
data = data.drop('long', axis=1)
data = data.drop('unix_time', axis=1)
data = data.drop('merch_lat', axis=1)
data = data.drop('merch_long', axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(data.corr(),annot=True).set_title('Correlation')

In [ ]:
y = data['is_fraud']
x = data.drop('is_fraud', axis=1) 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

model = LogisticRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
y_pred_proba = model.predict_proba(x_test)[:, 1]


accuracy = accuracy_score(y_test, y_pred)

print(accuracy)